# Swimming 4 × 100 metre mixed medley relay
## Optimial assigments
> The mixed relay event sees teams of four swimmers – two men and two women – racing against each other, with each team member swimming a 100m leg in one the four main swimming strokes (backstroke, breaststroke, butterfly and freestyle), one after the other. Teams can decide which athlete will swim each stroke.

In [1]:
import pandas as pd
import datetime
from ortools.graph.pywrapgraph import SimpleMinCostFlow

## The American's 5th place team result

* Backstroke: men’s 100 back bronze medalist Ryan Murphy
* Beaststroke: women’s 100 breast gold medalist Lydia Jacoby
* Butterfly: women’s 100 fly fourth-place finisher Torri Huske
* Freestyle: men’s 100 free gold medalist Caeleb Dressel
    
Final time: 3:40.58 (5/8)

In [2]:
men = [
    'Michael Andrew',
    'Zach Apple',
    'Hunter Armstrong',
    'Bowe Becker',
    'Gunnar Bentz',
    'Michael Brinegar',
    'Patrick Callan',
    'Brooks Curry',
    'Caeleb Dressel',
    'Nic Fink',
    'Bobby Finke',
    'Townley Haas',
    'Zach Harting',
    'Chase Kalisz',
    'Drew Kibler',
    'Jay Litherland',
    'Bryce Mefford',
    'Jake Mitchell',
    'Ryan Murphy',
    'Blake Pieroni',
    'Andrew Seliskar',
    'Tom Shields',
    'Kieran Smith',
    'Andrew Wilson'
]

women = [
    'Phoebe Bacon',
    'Erika Brown',
    'Claire Curzan',
    'Catie DeLoof',
    'Kate Douglass',
    'Hali Flickinger',
    'Brooke Forde',
    'Katie Grimes',
    'Natalie Hinds',
    'Torri Huske',
    'Lydia Jacoby',
    'Lilly King',
    'Annie Lazor',
    'Katie Ledecky',
    'Paige Madden',
    'Simone Manuel',
    'Katie McLaughlin',
    'Allison Schmitt',
    'Bella Sims',
    'Regan Smith',
    'Olivia Smoliga',
    'Erica Sullivan',
    'Alex Walsh',
    'Abbey Weitzeil',
    'Emma Weyant',
    'Rhyan White'
]

events = ['100 m backstroke', '100 m breaststroke', '100 m butterfly', '100 m freestyle']

qualifying_times = [
    ('100 m freestyle', 'Abbey Weitzeil', 53.53),
    ('100 m freestyle', 'Erika Brown', 53.59),
    ('100 m freestyle', 'Olivia Smoliga', 53.63),
    ('100 m backstroke', 'Regan Smith', 58.35),
    ('100 m backstroke', 'Rhyan White', 58.6),
    ('100 m backstroke', 'Olivia Smoliga', 58.72),
    ('100 m breaststroke', 'Lilly King', 64.79),
    ('100 m breaststroke', 'Lydia Jacoby', 65.28),
    ('100 m breaststroke', 'Annie Lazor', 65.6),
    ('100 m butterfly', 'Torri Huske', 55.66),
    ('100 m butterfly', 'Claire Curzan', 56.43),
    ('100 m butterfly', 'Kate Douglass', 56.56),
    ('100 m freestyle', 'Caeleb Dressel', 47.39),
    ('100 m freestyle', 'Zach Apple', 47.72),
    ('100 m freestyle', 'Blake Pieroni', 48.16),
    ('100 m backstroke', 'Ryan Murphy', 52.33),
    ('100 m backstroke', 'Hunter Armstrong', 52.48),
    ('100 m breaststroke', 'Michael Andrew', 58.73),
    ('100 m breaststroke', 'Andrew Wilson', 58.74),
    ('100 m breaststroke', 'Nic Fink', 58.8),
    ('100 m butterfly', 'Caeleb Dressel', 49.87),
    ('100 m butterfly', 'Tom Shields', 51.19)
]

In [3]:
pd.DataFrame(qualifying_times, columns=["event", "swimmer", "time (sec)"]).sort_values(["event", "time (sec)"])

,event,swimmer,time (sec)
15,100 m backstroke,Ryan Murphy,52.33
16,100 m backstroke,Hunter Armstrong,52.48
3,100 m backstroke,Regan Smith,58.35
4,100 m backstroke,Rhyan White,58.60
5,100 m backstroke,Olivia Smoliga,58.72
17,100 m breaststroke,Michael Andrew,58.73
18,100 m breaststroke,Andrew Wilson,58.74
19,100 m breaststroke,Nic Fink,58.80
6,100 m breaststroke,Lilly King,64.79
7,100 m breaststroke,Lydia Jacoby,65.28


In [4]:
# create nodes
id_ = 1
source = 0
swimmer_nodes = {}
event_nodes = {}
gender_nodes = {}

for g in ['men', 'women']:
    gender_nodes[g] = id_
    id_ += 1

for s in men + women:
    swimmer_nodes[s] = id_
    id_ += 1
    
for e in events:
    event_nodes[e] = id_
    id_ += 1

sink = id_

node_names = {}

for n, i in swimmer_nodes.items():
    node_names[i] = n
for n, i in event_nodes.items():
    node_names[i] = n
for n, i in gender_nodes.items():
    node_names[i] = n

In [5]:
SWIMMERS_PER_GENDER = 2

min_cost_flow = SimpleMinCostFlow()

# enforce 2 swimmers per gender constraint
for _, gender_node_id in gender_nodes.items():
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        source, 
        gender_node_id, 
        capacity=SWIMMERS_PER_GENDER, 
        unit_cost=0
    )
    
# assign swimmers by gender
for swimmer in men:
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        gender_nodes['men'], 
        swimmer_nodes[swimmer], 
        capacity=1, 
        unit_cost=0
    )
    
for swimmer in women:
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        gender_nodes['women'], 
        swimmer_nodes[swimmer], 
        capacity=1, 
        unit_cost=0
    )
    
# swimmer times
for event, swimmer, time in qualifying_times:
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        swimmer_nodes[swimmer], 
        event_nodes[event], 
        capacity=1, 
        unit_cost=int(time*100) # time in ms
    )

# enforce all events covered
for _, event_node_id in event_nodes.items():
    min_cost_flow.AddArcWithCapacityAndUnitCost(
        event_node_id, 
        sink, 
        capacity=1, 
        unit_cost=0
    )


# supplies
min_cost_flow.SetNodeSupply(source, 4)
min_cost_flow.SetNodeSupply(sink, -4)

for node_id in range(source+1, sink):
    min_cost_flow.SetNodeSupply(node_id, 0)

In [6]:
assert min_cost_flow.Solve() == min_cost_flow.OPTIMAL

In [7]:
print(f"Fastest time: {datetime.timedelta(seconds=min_cost_flow.OptimalCost() / 100)}")

Fastest time: 0:03:40.130000


# Optimal team

In [8]:
for arc in range(min_cost_flow.NumArcs()):
  # Can ignore arcs leading out of source or into sink.
  if min_cost_flow.Tail(arc)!=source and min_cost_flow.Head(arc)!=sink:
    # Arcs in the solution have a flow value of 1. Their start and end nodes
    # give an assignment of worker to task.
    if min_cost_flow.Flow(arc) > 0:
        swimmer_id = min_cost_flow.Tail(arc)
        event_id = min_cost_flow.Head(arc)
        time = min_cost_flow.UnitCost(arc)
        if time > 0:
            print(f"{node_names[swimmer_id]} -> {node_names[event_id]} ({time/100})")

Regan Smith -> 100 m backstroke (58.35)
Torri Huske -> 100 m butterfly (55.66)
Caeleb Dressel -> 100 m freestyle (47.39)
Michael Andrew -> 100 m breaststroke (58.73)


## Noted changes:
* Regan Smith replaces Ryan Murphy in backstroke
* Michael Andrew replaces Lydia Jacoby in breaststroke